<a href="https://colab.research.google.com/github/Segrey372017/gb_ML_in_business/blob/main/ssmirnov_ml_business_tp_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix, classification_report, log_loss

In [3]:
import dill, pickle

# Итоговый проект курса "Машинное обучение в бизнесе"

### Задание 1
<p> выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site</p>

### Описание набора данных 
<p> Данный набор данных, реализут задачу классификации. Поскольку целью курсового проекта является построение микросервиса, на основе данного набора можно построить прототип сервиса по экспресс скорингу пользователей на предмет риска возникновения сахарного диабета. Поскольку онлайн сервис не заменяет консультацию врача, данный датасет не содержит узкомедицинских факторов: результатов лабораторных анализов, результатов медицинских манипуляций, чуствительной для пациентов информации о состоянии здоровья. Тем немене высокий риск возникновения диабета должен стать сигналом для пользователя пройти углубленное обследование.</p>
<hr>
<h3> Подробное описание набора "Diabetes Health Indicators Dataset"</h3>

<h4> <a href= "https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset">Ссылка на Kaggle </a></h4>

<p>Diabetes is among the most prevalent chronic diseases in the United States, impacting millions of Americans each year and exerting a significant financial burden on the economy. Diabetes is a serious chronic disease in which individuals lose the ability to effectively regulate levels of glucose in the blood, and can lead to reduced quality of life and life expectancy. After different foods are broken down into sugars during digestion, the sugars are then released into the bloodstream. This signals the pancreas to release insulin. Insulin helps enable cells within the body to use those sugars in the bloodstream for energy. Diabetes is generally characterized by either the body not making enough insulin or being unable to use the insulin that is made as effectively as needed.

Complications like heart disease, vision loss, lower-limb amputation, and kidney disease are associated with chronically high levels of sugar remaining in the bloodstream for those with diabetes. While there is no cure for diabetes, strategies like losing weight, eating healthily, being active, and receiving medical treatments can mitigate the harms of this disease in many patients. Early diagnosis can lead to lifestyle changes and more effective treatment, making predictive models for diabetes risk important tools for public and public health officials.

The scale of this problem is also important to recognize. The Centers for Disease Control and Prevention has indicated that as of 2018, 34.2 million Americans have diabetes and 88 million have prediabetes. Furthermore, the CDC estimates that 1 in 5 diabetics, and roughly 8 in 10 prediabetics are unaware of their risk. While there are different types of diabetes, type II diabetes is the most common form and its prevalence varies by age, education, income, location, race, and other social determinants of health. Much of the burden of the disease falls on those of lower socioeconomic status as well.  </p>
<p> The Behavioral Risk Factor Surveillance System (BRFSS) is a health-related telephone survey that is collected annually by the CDC. Each year, the survey collects responses from over 400,000 Americans on health-related risk behaviors, chronic health conditions, and the use of preventative services. It has been conducted every year since 1984. For this project, a csv of the dataset available on Kaggle for the year 2015 was used. This original dataset contains responses from 441,455 individuals and has 330 features. These features are either questions directly asked of participants, or calculated variables based on individual participant responses.</p>

In [4]:
DATA_LINK = "https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_diabetes_012_health_indicators_BRFSS2015.csv"

In [5]:
df = pd.read_csv(DATA_LINK, sep = ',')

In [6]:
df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_012          253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

Набор содержит 20 признаков. Часть признаков общемедицинские - индекс массы тела, повышенное давление, наличие сердечно-сосудистых заболеваний и т.д. Другая часть признаков - социально демографические: пол, возраст, уровень образования, уровень дохода, наличие вредных привычек. Ряд признаков можно охарактеризовать как производные: общий уровень здоровья, уровень психического здоровья и т.д.

Большинство признакаов имеют бинарные значения 0 или 1, числовое значение. Ряд признаков, набример возраст закодированые номером группы. Для них Надо будет выполнять предобработку

In [8]:
df.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

Для специфики нашей задачи подойдет не весь набор признаков. Сосредочимся на тех, которе легко понятны для пользователя и последующей интерпритации:
<ul>
<li> Diabetes_012 - целевой признак: 0 - нет диабета, 1 - преддиабет, 2 диабет </li>
<li> HighBP - "ПовышенноеДавление": 0 - в норме; 1 - повышенное - </li>
<li> HighChol - "ВысокийХолестерин": 0 - нет; 1 - да </li>
<li> CholCheck - "ПроверкаХолистерина": Измерялся ли за последние 5 лет уровень холистерина (0 - нет, 1 - да) </li>
<li> BMI - "Индекс массы тела" - числовое значение: $масса / рост^2$ </li>
<li> Smoker - "Курильщик" 0 - нет; 1 - да (НЕОЖИДАННО: считается да если выкурили от 100 сигарет  В ТЕЧЕНИЕ  ЖИЗНИ</li>
<li> Stroke - "Инсульт" перенесли инсульт: 0 - нет; 1 - да</li>
<li> HeartDiseaseorAttack  - "Сердечный приступ (инфаркт)": 0-  нет 1 - да </li>
<li>PhysActivity - ФизическаяАктивность - была ли физическая активность за последние 30 дней за исключением работы (спорт, прогулки, пробежки) </li>
<li> Fruits - Съедаете ли вы хотя бы 1 фрукт в течение дня (0 - нет; 1 - да)</li>
<li>Veggies - тоже самое про овощи  </li>
<li> HvyAlcoholConsump - "Алкоголь" тяжелое употребление алкоголя: 1 - если выпиваете алкогольный напиток 14 раз в неделю (мужчины) 7 раз (женщины), реже 0.  </li>
<li> Sex - "Пол": 0 - для женщин, 1 - для мужчин</li>
<li> Age - "Возраст" - в наборе данных используется номер возрастной группы по 13 групповой возрасной классификации.</li>
</ul>
<hr>
<p> P.S. Надо будет подготовить функции для расчта индекса массы тела по массе и росту пользователя, для расчета возрастной групы по возрасту. </p>

In [9]:
sel_cols = ['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies','HvyAlcoholConsump', 'Sex', 'Age']

И так, все отобранные признаки категориальные (да/нет), один непрерывный числовой - Индекс массы тела. Таким образом получится простая в заполнении анкета.

In [10]:
# перенесем отобранные признаки в новый датафрейм
data = df[sel_cols]
data.head(3)

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,Sex,Age
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0


In [11]:
# набор данных должен быть без пропусков
data.isna().mean()*100

Diabetes_012            0.0
HighBP                  0.0
HighChol                0.0
CholCheck               0.0
BMI                     0.0
Smoker                  0.0
Stroke                  0.0
HeartDiseaseorAttack    0.0
PhysActivity            0.0
Fruits                  0.0
Veggies                 0.0
HvyAlcoholConsump       0.0
Sex                     0.0
Age                     0.0
dtype: float64

Поскольку планируется сервис по экспресс-тестированию. Целесообразно оставить в целевом признаке 'Diabetes_012' две категории 0 - отсутствует риск диабета; 1 - высокий риск диабета (объединим 1 и 2 классы)

In [12]:
data.Diabetes_012.loc[data.Diabetes_012 != 0] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
data.Diabetes_012.value_counts()

0.0    213703
1.0     39977
Name: Diabetes_012, dtype: int64

In [14]:
imbalance = 213703/39977
imbalance

5.345648748030118

Набор данный имеет сильный дисбаланс на 213 тысяч здоровых пригодится всего 40 тысяч людей с диабетом или риском развития диабета

Объявим функции для расчета необходимых данных: Индекса массы тела, Возрастной группы. <hr>

<h4> 13 ранговая таблица возрастов</h4>
<ol> 
<li>Age 18 - 24 </li>
<li>Age 25 to 29 </li>
<li>Age 30 to 34 </li>
<li>Age 35 to 39 </li>
<li>Age 40 to 44 </li>
<li>Age 45 to 49 </li>
<li>Age 50 to 54 </li>
<li>Age 55 to 59 </li>
<li>Age 60 to 64 </li>
<li>Age 65 to 69 </li>
<li>Age 70 to 74 </li>
<li>Age 75 to 79 </li>
<li>Age 80 or older </li>
</ol>

In [15]:
def get_bmi(weight, height):
    """
    Возвращает значение Индекса массы тела, рассчитываемый по формуле вес человека (кг)
    деленный на рост (в метрах) в квадрате 
    """
    return weight / height**2

def get_age_group(age):
    """
    По заданному значению возраста возращает 
    номер возрастной группы по 13 ранговой группировке
    """
    age_groups = {1:[18, 19, 20, 21, 22, 23, 24], #словарь с группами возрастов
                  2:[25, 26, 27, 28, 29],
                  3:[30, 31, 32, 33, 34],
                  4:[35, 36, 37, 38, 39],
                  5:[40, 41, 42, 43, 44],
                  6:[45, 46, 47, 48, 49],
                  7:[50, 51, 52, 53, 54],
                  8:[55, 56, 57, 58, 59],
                  9:[60, 61, 62, 63, 64],
                  10:[65, 66, 67, 68, 69],
                  11:[70, 71, 72, 73, 74],
                  12:[75, 76, 77, 78, 79],
                  13:[80, 81, 82, 83, 84]}

    #Выполним подгонку полученного возраста если меньше 18, то будет 18, если больше 84, ставим 84
    if age < 18:
        age = 18
    elif age > 84:
        age = 84
    
    for group in age_groups.keys(): # итерируемся по группам словаря
        if age in age_groups[group]:
            return group

    return

In [16]:
# базовая описательная статистика набора
data.describe()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,Sex,Age
count,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000
mean,0.157588,0.429001,0.424121,0.962670,28.382364,0.443169,0.040571,0.094186,0.756544,0.634256,0.811420,0.056197,0.440342,8.032119
std,0.364355,0.494934,0.494210,0.189571,6.608694,0.496761,0.197294,0.292087,0.429169,0.481639,0.391175,0.230302,0.496429,3.054220
min,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,1.000000,24.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,6.000000
50%,0.000000,0.000000,0.000000,1.000000,27.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,8.000000
75%,0.000000,1.000000,1.000000,1.000000,31.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,10.000000
max,1.000000,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000


Анализ средних и максимальных значений, значительных выбросов и некорректных значений не выявил.

In [17]:
# выделим колонки с признаками и тагертом
feats_col = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies','HvyAlcoholConsump', 'Sex', 'Age']
tagert = 'Diabetes_012'

In [18]:
# разбиваем набор данных на таргет и фичи
X = data[feats_col]
y = data[tagert]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 5)

In [20]:
# сохраним наборы в отдельные файлы
X_train.to_csv('tp_x_train.csv', index = None)
X_test.to_csv('tp_x_test.csv', index = None)
y_train.to_csv('tp_y_train.csv', index = None)
y_test.to_csv('tp_y_test.csv', index = None)


In [21]:
# загружаем соотвествующие наборы данных из файлов на гитхаб
"""
X_train = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_x_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_x_test.csv')

y_train = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_y_train.csv')
y_test = pd.read_csv('https://github.com/Segrey372017/gb_ML_in_business/blob/main/tp_y_test.csv')
"""

"\nX_train = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_x_train.csv')\nX_test = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_x_test.csv')\n\ny_train = pd.read_csv('https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/tp_y_train.csv')\ny_test = pd.read_csv('https://github.com/Segrey372017/gb_ML_in_business/blob/main/tp_y_test.csv')\n"

Поскольку данные в наборе уже предобработаны, собирать сложный пайплан нет необходимости, тем неменее для "чистоты" эксперимента отмаштабируем все наши данные в интервал от [0...1]. Понятно, что реальному изменению подвергнутся только признаки BMI и AGE

In [22]:
# Будем использвать два классификатора Машину опортных векторов и Легкий бустинг

svc_model = Pipeline([('min_max_scaler', MinMaxScaler((0,1))),
                      ('classifier', SVC(kernel= 'linear', class_weight= {0:imbalance, 1: 1}))]) # линейное ядро для сокращения рассчетов

lgbm_model = Pipeline([('min_max_scaler', MinMaxScaler((0,1))),
                      ('classifier', LGBMClassifier(n_estimators = 100, class_weight= {0:imbalance, 1:1}, reg_lambda= 0.0))])

In [23]:
 # тренируем модели
 # svc_model.fit(X_train, np.ravel(y_train.values))  модель не сошлась


In [24]:
%%time
lgbm_model.fit(X_train, np.ravel(y_train))

CPU times: user 9.25 s, sys: 66.7 ms, total: 9.32 s
Wall time: 8.55 s


Pipeline(steps=[('min_max_scaler', MinMaxScaler()),
                ('classifier',
                 LGBMClassifier(class_weight={0: 5.345648748030118, 1: 1}))])

In [25]:
y_lgbm_pred = lgbm_model.predict(X_test)

In [26]:
print(classification_report(y_test, y_lgbm_pred))

              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91     42741
         1.0       0.50      0.00      0.00      7995

    accuracy                           0.84     50736
   macro avg       0.67      0.50      0.46     50736
weighted avg       0.79      0.84      0.77     50736



In [27]:
# модель дала довольно слабые результаты классификации по первому классу
# попробуем выполнить поиск по сетке
gs_param = {'classifier__n_estimators' : [10, 30, 60, 100, 120, 200], # количество решающих деревьев
             'classifier__class_weight' : [None, {0:imbalance, 1:1}, {0:5, 1:1}, {0:4, 1:1}], # соотношение при дисбалансе
             'classifier__reg_lambda' : [0.0, 0.05, 0.1, 0.2, 0.3, 0.4]} # штрафы за переобучение


In [28]:
grid = GridSearchCV(lgbm_model,
                    param_grid=gs_param,
                    cv=5,
                    scoring = 'f1',
                    refit=False)


In [29]:
%%time
# grid.fit(X_train, np.ravel(y_train))

CPU times: user 0 ns, sys: 16 µs, total: 16 µs
Wall time: 1.25 ms


In [30]:
#grid.best_params_

In [31]:
# обучим модель с опитмальными параметрами
lgbm_model = Pipeline([('min_max_scaler', MinMaxScaler((0,1))),
                      ('classifier', LGBMClassifier(n_estimators = 200, class_weight= None, reg_lambda= 0.05))])

In [32]:
lgbm_model.fit(X_train, np.ravel(y_train))

Pipeline(steps=[('min_max_scaler', MinMaxScaler()),
                ('classifier',
                 LGBMClassifier(n_estimators=200, reg_lambda=0.05))])

In [33]:
y_lgbm_pred = lgbm_model.predict(X_test)

In [34]:
print(classification_report(y_test, y_lgbm_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92     42741
         1.0       0.56      0.14      0.22      7995

    accuracy                           0.85     50736
   macro avg       0.71      0.56      0.57     50736
weighted avg       0.81      0.85      0.81     50736



In [35]:
# пвыполним подгонку порога вероятностей
y_lgbm_proba = pd.Series(lgbm_model.predict_proba(X_test)[:, 1]) # вероятности для певого класса

In [36]:
# функция для получени метрик модели и подбора наилучшего порога
def get_treshold(y_test, y_score, b = 1):
    """
    Функция для подбора оптимального порога вероятности.
    Максимизирует F-score
    """
    
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 +b**2) * (precision * recall) / (b**2 * precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    auc = roc_auc_score(y_test, y_score)
    # логарифм потерь
    llos = log_loss(y_test, y_score)
    # отдадим в виде словаря порог вероятности и метрики
    return {'BestTreshold': thresholds[ix], 'Precision': precision[ix], 'Recall': recall[ix], 'ROC_AUC': auc, 'F-Score':fscore[ix],'log-loss': llos}

In [37]:
res = get_treshold(y_test, y_lgbm_proba) # автоматически порог подобрать не удалось
res

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


{'BestTreshold': 0.8445551875457856,
 'F-Score': nan,
 'Precision': 0.0,
 'ROC_AUC': 0.7965799645578187,
 'Recall': 0.0,
 'log-loss': 0.35681252295647153}

In [38]:
print(classification_report(y_test, y_lgbm_proba > 0.42)) # выберем порог, дающий удовлетворительную точность и полноту

              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     42741
         1.0       0.51      0.25      0.34      7995

    accuracy                           0.84     50736
   macro avg       0.69      0.60      0.62     50736
weighted avg       0.81      0.84      0.82     50736



In [39]:
treshold = 0.42

Сохраним полученную модель в файл

In [40]:
with open("tp_lgbm_model.dill", "wb") as f:
    dill.dump(lgbm_model, f)

In [41]:
# также сохраним найденный порог вероятностей
with open('tp_treshold.pkl', "wb") as f:
    pickle.dump(treshold, f)

### Задание 2
<p> Реализовать ноутбук с сервером</p>

In [42]:
! pip install flask-ngrok

In [43]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

ngrok config add-authtoken 28I4sdcpBe4UB4hjpmYyH3yLvvt_6Gc3WEvrCtS9KFfL9dC1Q


In [44]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 28I4sdcpBe4UB4hjpmYyH3yLvvt_6Gc3WEvrCtS9KFfL9dC1Q
!./ngrok http 80

--2022-04-28 12:43:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2022-04-28 12:43:50 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [45]:
# Запуск сервера на Flask

def main():
    """
    Выполняет загрузку и настройку модели машинного обучения
    """
    model_path = '/content/tp_lgbm_model.dill'
    tr_path =  '/content/tp_treshold.pkl'

    model = None
    treshold = None

    with open(model_path, 'rb') as f:
        model = dill.load(f)

    with open(tr_path, 'rb') as f:
        treshold = pickle.load(f)
    
    return model, treshold

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/", methods = ['GET'])
def hello():
    return "<h3> Hello! Test Flask Server running <h3>"


@app.route("/predict", methods = ["POST"])
def predict():
    resp = {"OK": False} # слоарь для формирования ответа от сервера
    # создадим структуру для хранения данных поступившего запроса
    feats_col = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies','HvyAlcoholConsump', 'Sex', 'Age']
    data = dict()
    for col in feats_col:
        data[col] = ""

    request_json = dict(request.get_json()) # читаем данные поступившие от клиента
    print("полученный запрос", request_json)

    for col in feats_col:
        #print('Колонка', col)
        if request_json[col] != '':
            data[col] = request_json[col]
 
    data = pd.DataFrame(data = data, columns = feats_col, index = [0])
    #print("Данные для предсказания", data)

    pred = model.predict_proba(data)
    print("Рузультат предсказания",pred)
    resp['proba'] = pred[:, 1][0]
    resp['treshold'] = treshold
    resp['OK'] = True

    print('Ответ сервера', resp)

    return jsonify(resp)

if __name__ == '__main__':
    model, treshold = main()
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eb85-34-91-252-59.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Apr/2022 12:44:39] "POST /predict HTTP/1.1" 200 -


полученный запрос {'Age': 4, 'BMI': 27.757487216946675, 'CholCheck': 1, 'Fruits': 0, 'HeartDiseaseorAttack': 0, 'HighBP': 1, 'HighChol': 0, 'HvyAlcoholConsump': 0, 'PhysActivity': 1, 'Sex': 1, 'Smoker': 0, 'Stroke': 0, 'Veggies': 1}
Рузультат предсказания [[0.96331221 0.03668779]]
Ответ сервера {'OK': True, 'proba': 0.03668778752920068, 'treshold': 0.42}


...

полученный запрос {'HighBP': 0.0, 'HighChol': 0.0, 'CholCheck': 1.0, 'BMI': 26.0, 'Smoker': 1.0, 'Stroke': 0.0, 'HeartDiseaseorAttack': 0.0, 'PhysActivity': 1.0, 'Fruits': 1.0, 'Veggies': 1.0, 'HvyAlcoholConsump': 1.0, 'Sex': 0.0, 'Age': 8.0}
Рузультат предсказания [[0.98963405 0.01036595]]
Ответ сервера {'OK': True, 'proba': 0.010365948979538958, 'treshold': 0.42}


127.0.0.1 - - [28/Apr/2022 12:51:21] "POST /predict HTTP/1.1" 200 -


полученный запрос {'HighBP': 1.0, 'HighChol': 1.0, 'CholCheck': 1.0, 'BMI': 29.0, 'Smoker': 0.0, 'Stroke': 0.0, 'HeartDiseaseorAttack': 0.0, 'PhysActivity': 1.0, 'Fruits': 1.0, 'Veggies': 1.0, 'HvyAlcoholConsump': 0.0, 'Sex': 0.0, 'Age': 8.0}
Рузультат предсказания [[0.75905471 0.24094529]]
Ответ сервера {'OK': True, 'proba': 0.24094529185152713, 'treshold': 0.42}


127.0.0.1 - - [28/Apr/2022 12:51:22] "POST /predict HTTP/1.1" 200 -


полученный запрос {'HighBP': 0.0, 'HighChol': 1.0, 'CholCheck': 1.0, 'BMI': 29.0, 'Smoker': 1.0, 'Stroke': 0.0, 'HeartDiseaseorAttack': 0.0, 'PhysActivity': 1.0, 'Fruits': 1.0, 'Veggies': 1.0, 'HvyAlcoholConsump': 0.0, 'Sex': 1.0, 'Age': 13.0}
Рузультат предсказания [[0.8216824 0.1783176]]
Ответ сервера {'OK': True, 'proba': 0.1783175998241419, 'treshold': 0.42}
